In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam

In [3]:
batch_size = 32
img_size = (224, 224)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)


train_ds = train_datagen.flow_from_directory(
    './archive/Training',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_ds = train_datagen.flow_from_directory(
    './archive/Training',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_ds = test_datagen.flow_from_directory(
    './archive/Testing',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [6]:
for layer in base_model.layers[-6:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))  # 4 output classes

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

#model.compile(optimizer=Adam(learning_rate=lr_schedule(0)), loss='categorical_crossentropy', metrics=['accuracy'])

#lr_scheduler = LearningRateScheduler(lr_schedule)

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21138500 (80.64 MB)
Trainable params: 15863044 (60.51 MB)
Non-trainable params: 5275456 (20.12 MB)
_________________________________________________________________


In [9]:
epochs = 15  # adjust as needed
history = model.fit(
    train_ds,
    steps_per_epoch=train_ds.samples // 32,
    epochs=epochs,
    validation_data=val_ds,
    validation_steps=val_ds.samples // 32
)

Epoch 1/15
142/142 [==============================] - 2361s 16s/step - loss: 0.6348 - accuracy: 0.7480 - val_loss: 0.7776 - val_accuracy: 0.7143
Epoch 2/15
142/142 [==============================] - 1502s 11s/step - loss: 0.3083 - accuracy: 0.8934 - val_loss: 0.4598 - val_accuracy: 0.8464
Epoch 3/15
142/142 [==============================] - 1500s 11s/step - loss: 0.2403 - accuracy: 0.9152 - val_loss: 0.4214 - val_accuracy: 0.8634
Epoch 4/15
142/142 [==============================] - 1499s 11s/step - loss: 0.1835 - accuracy: 0.9374 - val_loss: 0.4105 - val_accuracy: 0.8830
Epoch 5/15
142/142 [==============================] - 1504s 11s/step - loss: 0.1495 - accuracy: 0.9489 - val_loss: 0.4436 - val_accuracy: 0.8625
Epoch 6/15
142/142 [==============================] - 1501s 11s/step - loss: 0.1219 - accuracy: 0.9564 - val_loss: 0.3012 - val_accuracy: 0.9232
Epoch 7/15
142/142 [==============================] - 1500s 11s/step - loss: 0.1321 - accuracy: 0.9577 - val_loss: 0.3113 - val_ac

In [10]:
test_loss, test_acc = model.evaluate(test_ds)
print(f'Test Accuracy: {test_acc * 100:.2f}%')  

41/41 [==============================] - 583s 14s/step - loss: 0.1663 - accuracy: 0.9657
Test Accuracy: 96.57%


In [17]:
from keras.models import load_model 
model.save("model1.h5") 
loaded_model = load_model("model1.h5") 
loss, accuracy = loaded_model.evaluate(test_ds) 

41/41 [==============================] - 1076s 26s/step - loss: 0.1663 - accuracy: 0.9657


In [16]:
# Saving model structure to a JSON file 
model_json = model.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model_json)
model_json = model.to_json() # with open("network.json", "w") as json_file: 
    json_file.write(model_json) 

# Saving weights of the model to a HDF5 file 
model.save_weights("model2.h5") 

# Loading JSON file 
json_file = open("network.json", 'r') 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json) 

# Loading weights 
loaded_model.load_weights("network.h5") 
loss, accuracy = loaded_model.evaluate(test_ds) 


SyntaxError: invalid syntax (255496528.py, line 3)

In [14]:
# Saving model structure to a YAML file 
model_yaml = model.to_yaml() 
with open("network.yaml", "w") as yaml_file: 
	yaml_file.write(model_yaml) 

# Saving weights of the model to a HDF5 file 
model.save_weights("network.h5") 

# Loading YAML file 
yaml_file = open("network.yaml", 'r') 
loaded_model_yaml = yaml_file.read() 
yaml_file.close() 
loaded_model = model_from_yaml(loaded_model_yaml) 

# Loading weights 
loaded_model.load_weights("network.h5") 
loss, accuracy = loaded_model.evaluate(test_ds) 


RuntimeError: Method `model.to_yaml()` has been removed due to security risk of arbitrary code execution. Please use `model.to_json()` instead.